# Фреймворк PyTorch для разработки искусственных нейронных сетей
# Урок 8. Generative Adversarial Networks

# <font color='red'>Практическое задание</font>

### Домашнее задание к уроку 8

Переписать загрузку данных с python функций на Dataset и Dataloader и применить сеть с attention

Выполнил ___Соковнин ИЛ___

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import pandas as pd

In [2]:
epochs = 30
num_samples = 10000
data_path = './data/fra-eng/fra.txt'

In [3]:
class eng_fra_dataset(torch.utils.data.Dataset):
    def __init__(self, file_name=data_path, num_samples=num_samples, nrows=num_samples):
        
        input_vocab = set()
        output_vocab = set()

        print(f'Загружаем файл: {file_name}')
        df = pd.read_csv(data_path, sep = '\t', header = None, nrows=num_samples)
        df.rename(columns = {0 : 'eng', 1: 'fra'}, inplace = True)
        # df = df[['eng', 'fra']]
        
        input_texts = list(df['eng'])
        target_texts = list(df['fra'])
        texts = df[['eng','fra']].values.tolist()

        for input_text in input_texts:
            for word in input_text.split():
                input_vocab.add(word.strip())

        for target_text in target_texts:
            for word in target_text.split():
                output_vocab.add(word.strip())

        output_vocab = set(list(output_vocab))

        input_vocab2index = {word: i+2 for i, word in enumerate(input_vocab)}
        output_vocab2index = {word: i+2 for i, word in enumerate(output_vocab)}   
        
        self.texts = texts
        self.input_texts = input_texts
        self.target_texts = target_texts
        self.input_vocab2index = input_vocab2index
        self.output_vocab2index = output_vocab2index
        
        print('Файл {file_name} загружен')
        
        
    def __len__(self):
        return len(self.input_vocab)
    
    def __getitem__(self, idx):
        return self.texts[idx]
    
    def print_ds(self):
        for i in self.input_vocab2index:
            if self.input_vocab2index[i] < 12:
                print(i, self.input_vocab2index[i])


In [4]:
def indexesFromSentence(sentence, vocab):
    return [vocab.get(word.strip(), 0) for word in sentence.split(' ')]

def tensorFromSentence(sentence, vocab):
    indexes = indexesFromSentence(sentence, vocab)
    indexes.append(1)
    return torch.tensor(indexes, dtype=torch.long).view(-1, 1)


def tensorsFromSent(input_sentences, output_sentences, input_vocab2index, output_vocab2index):
    input_tensor = tensorFromSentence(input_sentences, input_vocab2index)
    target_tensor = tensorFromSentence(output_sentences, output_vocab2index)
    return (input_tensor, target_tensor)

In [5]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size)


class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=10):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        #output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size)

In [6]:
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=10):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[0]])

    decoder_hidden = encoder_hidden

    for di in range(target_length):
        decoder_output, decoder_hidden, decoder_attention = decoder(
            decoder_input, decoder_hidden, encoder_outputs)
        topv, topi = decoder_output.topk(1)
        decoder_input = topi.squeeze().detach()  # detach from history as input

        loss += criterion(decoder_output, target_tensor[di])
        if decoder_input.item() == 1:
            break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [7]:
ds = eng_fra_dataset()
# ds.print_ds()
ds[1]

Загружаем файл: ./data/fra-eng/fra.txt
Файл {file_name} загружен


['Hi.', 'Salut !']

In [8]:
encoder = EncoderRNN(len(ds.input_vocab2index)+2, 30)
attn_decoder1 = AttnDecoderRNN(30, len(ds.output_vocab2index)+2, dropout_p=0.1)

encoder_optimizer = torch.optim.SGD(encoder.parameters(), lr=0.01)
decoder_optimizer = torch.optim.SGD(attn_decoder1.parameters(), lr=0.01)
training_pairs = np.random.randint(0, len(ds.input_texts), size=10000)
criterion = nn.NLLLoss()

print_loss_total = 0

for i in range(1000):
    input_tensor, target_tensor = tensorsFromSent(ds.input_texts[training_pairs[i]], ds.target_texts[training_pairs[i]], \
                                                  ds.input_vocab2index, ds.output_vocab2index)

    loss = train(input_tensor, target_tensor, encoder,
               attn_decoder1, encoder_optimizer, decoder_optimizer, criterion)
    print_loss_total += loss
    
    print_loss_avg = print_loss_total / 1
    print_loss_total = 0
    print('(%d %d%%) %.4f' % (i, i / 10 * 100, print_loss_avg))


(0 0%) 8.6540
(1 10%) 8.6432
(2 20%) 8.7114
(3 30%) 8.7780
(4 40%) 8.6563
(5 50%) 8.6907
(6 60%) 8.6709
(7 70%) 8.6171
(8 80%) 8.7057
(9 90%) 8.6936
(10 100%) 8.5683
(11 110%) 8.5469
(12 120%) 8.7139
(13 130%) 8.5683
(14 140%) 8.5551
(15 150%) 8.5791
(16 160%) 8.7090
(17 170%) 8.6746
(18 180%) 8.5785
(19 190%) 8.5545
(20 200%) 8.5442
(21 210%) 8.6220
(22 220%) 8.4640
(23 229%) 8.4238
(24 240%) 8.4129
(25 250%) 8.6544
(26 260%) 8.5125
(27 270%) 8.6281
(28 280%) 8.4944
(29 290%) 8.5364
(30 300%) 4.3646
(31 310%) 2.8673
(32 320%) 3.4719
(33 330%) 5.6825
(34 340%) 6.9881
(35 350%) 3.6989
(36 360%) 5.7402
(37 370%) 8.4526
(38 380%) 6.3972
(39 390%) 2.8499
(40 400%) 5.6791
(41 409%) 3.5371
(42 420%) 6.9641
(43 430%) 3.4441
(44 440%) 2.9019
(45 450%) 3.5218
(46 459%) 8.5855
(47 470%) 3.4459
(48 480%) 3.3896
(49 490%) 5.7799
(50 500%) 8.3430
(51 509%) 4.1402
(52 520%) 8.3600
(53 530%) 3.4874
(54 540%) 3.4993
(55 550%) 5.6532
(56 560%) 8.3228
(57 570%) 8.2857
(58 580%) 3.3908
(59 590%) 3.4761
(

(463 4630%) 4.3882
(464 4640%) 2.5619
(465 4650%) 3.1845
(466 4660%) 3.5118
(467 4670%) 3.7234
(468 4680%) 4.5972
(469 4690%) 4.2440
(470 4700%) 5.6884
(471 4710%) 3.2573
(472 4720%) 4.5277
(473 4730%) 4.7703
(474 4740%) 3.5489
(475 4750%) 1.6232
(476 4760%) 4.6424
(477 4770%) 5.2152
(478 4780%) 2.1379
(479 4790%) 4.4938
(480 4800%) 3.1150
(481 4810%) 4.2398
(482 4820%) 3.1490
(483 4830%) 2.7097
(484 4840%) 2.2471
(485 4850%) 3.2650
(486 4860%) 2.9194
(487 4870%) 4.7739
(488 4880%) 2.2859
(489 4890%) 4.3408
(490 4900%) 4.5840
(491 4910%) 5.3370
(492 4920%) 4.5242
(493 4930%) 4.1507
(494 4940%) 5.4935
(495 4950%) 3.4002
(496 4960%) 3.6655
(497 4970%) 2.0248
(498 4980%) 3.8510
(499 4990%) 4.2003
(500 5000%) 1.5793
(501 5010%) 2.1432
(502 5020%) 1.8643
(503 5030%) 2.7311
(504 5040%) 4.2268
(505 5050%) 2.7305
(506 5060%) 3.2924
(507 5070%) 3.4501
(508 5080%) 2.6632
(509 5090%) 1.7728
(510 5100%) 2.4792
(511 5110%) 4.5720
(512 5120%) 2.8095
(513 5130%) 3.0489
(514 5140%) 6.3393
(515 5150%) 

(895 8950%) 2.3273
(896 8960%) 2.0332
(897 8970%) 2.3997
(898 8980%) 4.1392
(899 8990%) 1.6401
(900 9000%) 3.3084
(901 9010%) 2.1375
(902 9020%) 1.0310
(903 9030%) 3.6959
(904 9040%) 2.8701
(905 9050%) 2.0204
(906 9060%) 3.8856
(907 9070%) 4.8772
(908 9080%) 3.2336
(909 9090%) 1.0065
(910 9100%) 2.8203
(911 9110%) 6.7024
(912 9120%) 4.3834
(913 9130%) 5.5559
(914 9140%) 3.9844
(915 9150%) 4.4662
(916 9160%) 1.2627
(917 9170%) 1.8317
(918 9180%) 2.9188
(919 9190%) 2.0300
(920 9200%) 2.3838
(921 9210%) 4.1008
(922 9220%) 4.0528
(923 9230%) 4.3405
(924 9240%) 4.5046
(925 9250%) 2.3615
(926 9260%) 3.2252
(927 9270%) 2.0133
(928 9280%) 2.0388
(929 9290%) 2.2498
(930 9300%) 2.5454
(931 9310%) 2.5528
(932 9320%) 2.5090
(933 9330%) 2.5109
(934 9340%) 2.2407
(935 9350%) 3.1549
(936 9360%) 3.5790
(937 9370%) 1.5973
(938 9380%) 3.2621
(939 9390%) 2.3939
(940 9400%) 4.7931
(941 9410%) 3.3666
(942 9420%) 4.1133
(943 9430%) 4.9498
(944 9440%) 5.9166
(945 9450%) 2.9350
(946 9460%) 2.1426
(947 9470%) 